In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv("data/returns.csv")
funds = pd.read_excel("data/selected_etfs.xlsx")
all_funds = pd.read_excel("data/AllFunds.xlsx")

In [3]:
# Set parameters
benchmark = 0  # Benchmark return (0%)
confidence_level = 0.95  # 95% confidence level

# 1.  Conditional VaR (CVaR)
def calculate_cvar(returns, confidence_level):
    var = np.nanpercentile(returns, 100 * (1 - confidence_level))
    cvar = returns[returns < var].mean()
    return cvar

# 2. Lower Partial Moment (LPM)
def calculate_lpm(returns, benchmark, order=2):
    below_benchmark = returns[returns < benchmark]
    lpm = ((benchmark - below_benchmark) ** order).mean()
    return lpm

# 3. Omega Ratio
def calculate_omega(returns, benchmark):
    gains = returns[returns > benchmark].sum()
    losses = (benchmark - returns[returns < benchmark]).sum()
    return gains / losses if losses != 0 else np.inf


# 4. Conditional Drawdown (CDaR)
def calculate_cdar(returns, confidence_level):
    cumulative_returns = (1 + returns).cumprod()
    drawdowns = cumulative_returns / cumulative_returns.cummax() - 1
    var_drawdown = np.nanpercentile(drawdowns, 100 * (1 - confidence_level))
    cdar = drawdowns[drawdowns < var_drawdown].mean()
    return cdar

# 6. Downside Deviation
def calculate_downside_deviation(returns, benchmark):
    below_benchmark = returns[returns < benchmark]
    downside_deviation = np.sqrt(((benchmark - below_benchmark) ** 2).mean())
    return downside_deviation


In [4]:
# Daily
funds_list = list(funds["crsp_fundno"])
daily_summary = pd.DataFrame(columns=["Fund Name","Skewness","Kurtosis", "CVaR","LPM","Omega Ratio","CDaR","Downside Deviation"], index=funds_list)
for fund in funds_list:
    returns = all_funds.loc[all_funds["Fund Identifier"]==fund,"Daily Return per share"]
    daily_summary.loc[fund,"Fund Name"]=funds.loc[funds["crsp_fundno"]==fund, "fund_name"].iloc[0]
    daily_summary.loc[fund,"Skewness"]=returns.skew()
    daily_summary.loc[fund,"Kurtosis"]=returns.kurtosis()
    daily_summary.loc[fund,"CVaR"]= calculate_cvar(returns, confidence_level)
    daily_summary.loc[fund,"LPM"]=calculate_lpm(returns, benchmark)
    daily_summary.loc[fund,"Omega Ratio"]=calculate_omega(returns, benchmark)
    daily_summary.loc[fund,"CDaR"]=calculate_cdar(returns, confidence_level)
    daily_summary.loc[fund,"Downside Deviation"]=calculate_downside_deviation(returns, benchmark)
daily_summary


,Fund Name,Skewness,Kurtosis,CVaR,LPM,Omega Ratio,CDaR,Downside Deviation
27635,SPDR S&P 500 ETF Trust,-0.483367,12.181451,-0.026337,0.000133,1.172542,-0.191829,0.011547
31466,Vanguard Index Funds: Vanguard Total Stock Mar...,-0.555962,11.808323,-0.027005,0.000142,1.165121,-0.204422,0.011922
24697,"Invesco QQQ Trust, Series 1",-0.332831,6.378666,-0.03123,0.000191,1.182537,-0.288338,0.013827
35856,Vanguard Tax-Managed Funds: Vanguard Developed...,-0.689659,9.927164,-0.026327,0.000139,1.081346,-0.237044,0.0118
31464,Vanguard Index Funds: Vanguard Value Index Fun...,-0.49477,13.839747,-0.025227,0.000124,1.156534,-0.175623,0.011136
31250,Vanguard Bond Index Funds: Vanguard Total Bond...,-0.146441,3.554213,-0.006296,0.000008,1.108488,-0.152679,0.002854
16467,iShares Trust: iShares Core S&P Small-Cap ETF,-0.423666,7.978129,-0.031942,0.000209,1.116382,-0.260629,0.014462
16437,iShares Trust: iShares Core S&P Mid-Cap ETF,-0.611066,11.147717,-0.029834,0.00018,1.129787,-0.217538,0.013415
56420,"iShares, Inc: iShares Core MSCI Emerging Marke...",-0.419025,4.795171,-0.022552,0.000106,1.057174,-0.313479,0.010282
29218,SPDR Gold Trust: SPDR Gold Shares,-0.383999,5.200576,-0.022736,0.000098,1.079721,-0.419108,0.009905


In [5]:
# Monthly
funds_list = list(funds["crsp_fundno"])
monthly_summary = pd.DataFrame(columns=["Fund Name","Skewness","Kurtosis", "CVaR","LPM","Omega Ratio","CDaR","Downside Deviation"], index=funds_list)
for fund in funds_list:
    returns = pd.to_numeric(data.loc[data["crsp_fundno"]==fund,"mret"],errors='coerce')
    monthly_summary.loc[fund,"Fund Name"]=funds.loc[funds["crsp_fundno"]==fund, "fund_name"].iloc[0]
    monthly_summary.loc[fund,"Skewness"]=returns.skew()
    monthly_summary.loc[fund,"Kurtosis"]=returns.kurtosis()
    monthly_summary.loc[fund,"CVaR"]= calculate_cvar(returns, confidence_level)
    monthly_summary.loc[fund,"LPM"]=calculate_lpm(returns, benchmark)
    monthly_summary.loc[fund,"Omega Ratio"]=calculate_omega(returns, benchmark)
    monthly_summary.loc[fund,"CDaR"]=calculate_cdar(returns, confidence_level)
    monthly_summary.loc[fund,"Downside Deviation"]=calculate_downside_deviation(returns, benchmark)
monthly_summary

,Fund Name,Skewness,Kurtosis,CVaR,LPM,Omega Ratio,CDaR,Downside Deviation
27635,SPDR S&P 500 ETF Trust,-0.375057,0.539019,-0.086042,0.002193,2.018235,-0.178029,0.046831
31466,Vanguard Index Funds: Vanguard Total Stock Mar...,-0.373308,0.77107,-0.09005,0.002347,1.973925,-0.189697,0.048445
24697,"Invesco QQQ Trust, Series 1",-0.202172,0.06143,-0.092898,0.0023,2.136552,-0.274931,0.047961
35856,Vanguard Tax-Managed Funds: Vanguard Developed...,-0.220934,0.827276,-0.103324,0.00226,1.390238,-0.22341,0.047535
31464,Vanguard Index Funds: Vanguard Value Index Fun...,-0.374804,1.176139,-0.085399,0.001879,1.897244,-0.159417,0.043346
31250,Vanguard Bond Index Funds: Vanguard Total Bond...,-0.155828,1.772662,-0.028534,0.000151,1.59482,-0.151352,0.012288
16467,iShares Trust: iShares Core S&P Small-Cap ETF,-0.297431,1.5206,-0.11092,0.00338,1.655699,-0.242435,0.058141
16437,iShares Trust: iShares Core S&P Mid-Cap ETF,-0.408825,1.503074,-0.10576,0.002831,1.740607,-0.192202,0.053205
56420,"iShares, Inc: iShares Core MSCI Emerging Marke...",-0.068614,0.964116,-0.091676,0.00221,1.273291,-0.294301,0.047011
29218,SPDR Gold Trust: SPDR Gold Shares,-0.028366,0.291261,-0.086498,0.001736,1.376478,-0.394609,0.041669


In [6]:
# Standardize metrics in monthly summary
metrics_to_standardize = ["Skewness", "Kurtosis", "CVaR", "LPM", "Omega Ratio", "CDaR", "Downside Deviation"]

for metric in metrics_to_standardize:
    mean = monthly_summary[metric].mean()
    std = monthly_summary[metric].std()
    monthly_summary[f"{metric}_standardized"] = (monthly_summary[metric] - mean) / std

# Display standardized metrics
monthly_summary[[col for col in monthly_summary.columns if "standardized" in col]]


,Skewness_standardized,Kurtosis_standardized,CVaR_standardized,LPM_standardized,Omega Ratio_standardized,CDaR_standardized,Downside Deviation_standardized
27635,-0.787034,-0.604873,0.024884,0.175958,1.125926,0.756365,0.256405
31466,-0.779494,-0.195191,-0.155851,0.359023,0.984583,0.629903,0.39068
24697,-0.041486,-1.448046,-0.284255,0.303477,1.503336,-0.293908,0.350415
35856,-0.122397,-0.095961,-0.7543,0.255056,-0.877277,0.2645,0.314981
31464,-0.785943,0.519949,0.053865,-0.198117,0.739985,0.958085,-0.033487
31250,0.158368,1.573099,2.617762,-2.254896,-0.224693,1.045499,-2.617053
16467,-0.452283,1.128088,-1.096812,1.58918,-0.030501,0.058299,1.197233
16437,-0.932654,1.097146,-0.864172,0.93498,0.240339,0.602748,0.786643
56420,0.534469,0.145627,-0.229137,0.196023,-1.250317,-0.503843,0.271347
29218,0.708032,-1.042284,0.004329,-0.367816,-0.921167,-1.591033,-0.172981
